# HDFS

HDFS is het distributed file system van Hadoop dat de basis vormt voor een breed gamma van applicaties, waaronder MapReduce.
Dit framework maakt het mogelijk om niet-gespecialiseerde hardware te gebruiken om eenvoudig datacenters op te zetten of rekenclusters te beheren.
HDFS bereikt dit doel op een aantal manieren:
* Ten eerste wordt een file opgedeeld in verschillende blokken. Deze worden verdeeld over verschillende computers zodat code meerdere delen van het bestand kan gebruiken in parallel om zo de benodigde rekenkracht te verdelen over meerdere toestellen.
* Daarnaast worden de blokken ook gedupliceerd om zo fout-toleranter te zijn in het geval van een crash (hardware of software), stroomstoring, netwerkonderbreking.

Om dit te bereiken gebruikt Hadoop een Master-Slave architectuur dat bestaat uit een enkele namenode (master) en meerdere datanodes (slaves).
De namenode houdt bij hoeveel datanodes er actief zijn, welke blokken ze hebben en welke blokken bij welke file horen.
Indien er een datanode crasht gaat deze server dit detecteren en dit oplossen door de nodige blokken te kopieren van een andere datanode zodat er steeds voldoende kopies in het systeem aanwezig zijn.
Bij elke actie die uitgevoerd moet worden in het HDFS moet er steeds gevraagd worden aan de namenode welke blokken op welke datanodes we nodig hebben voor de gewenste file uit te lezen of code voor uit te voeren.
Het is dus duidelijk dat deze namenode een single-point-of-failure is wat ideaal is voor de availability van de cluster.
Dit kan opgelost worden door HDFS te runnen in een high-availability mode wat ervoor zorgt dat er steeds een backup aanwezig is voor de namenode die zeer snel de werking kan overnemen.
Deze structuur maakt het eenvoudig om aan horizontal scaling te doen door extra servers toe te voegen zonder dat er downtime is voor de hele cluster.

Dit resulteer in de volgende kenmerken van HDFS:
* High Throughput
* Scalability
* High Availability
* Data Reliability
* Fault Tolerance

## Starten en stoppen van de cluster

De cluster kan gestart worden door alle docker containers te starten die gedefinieerd worden in de yaml-file.
Dit kan via de docker desktop gui of via de command line door middel van het docker-compose commando.
Stoppen kan dan via dezelfde methodes

## Communiceren met het HDFS

Er zijn verschillende manieren om dit distributed bestandssysteem te gebruiken.
Ten eerste kan je gebruik maken van een command line interface (CLI) om bestanden uit te lezen, op te laden, ...
Daarnaast zijn er ook wrappers voor verschillende talen die toelaten om rechtstreeks vanuit code te interageren met het HDFS.
De voordelen van deze wrappers over een CLI zijn:
* Flexibiler om te interageren in applicaties.
* Gebruiksvriendelijker en gemakkelijker om te automatiseren.
* Eenvoudiger in onderhoud en te debuggen
* Volledige functionaliteit van een programmeertaal kan gebruikt worden zoals OO.

### Instantiering van een client

Veel verschillende talen beschikken over een wrapper om te communiceren met een HDFS.
In deze cursus gaan we gebruik maken van [InsecureClient in hdfscli](https://hdfscli.readthedocs.io/en/latest/quickstart.html) wrapper.
De eerste stap in het gebruik van de wrapper is te bepalen hoe de namenode van het hdfs gevonden kan worden.
Dit gebeurt als volgt:

In [1]:
from hdfs import InsecureClient

# maak de connectie met het hdfs
client = InsecureClient('http://localhost:9870', 'bigdata')
# localhost:9870 is de UI van het hdfs (hoe kan ik aan de namenode)
# bigdata is de user die geconfigureerd is op het hdfs

client.status('/')

{'accessTime': 0,
 'blockSize': 0,
 'childrenNum': 2,
 'fileId': 16385,
 'group': 'supergroup',
 'length': 0,
 'modificationTime': 1740043646546,
 'owner': 'root',
 'pathSuffix': '',
 'permission': '755',
 'replication': 0,
 'snapshotEnabled': True,
 'storagePolicy': 0,
 'type': 'DIRECTORY'}

### Aanmaken van files en directories

Om nu bestanden en folders aan te maken op dit distributed file systeem kunnen onderstaande functies gebruikt worden.

In [8]:
# make directories
if client.status('hdfs456', strict=False) is None: # standaard throwed status een error als de file niet bestaat, met strict=False returned het None
    client.makedirs('hdfs456')

# hier geen /user/bigdata nodig omdat we zijn ingelogd als de bigdata user -> standaard zitten in zijn directory namelijk /user/bigdata

# exists: hdfs dfs -test -d hdfs_path
# create: hdfs dfs -mkdir -p /bigdata/HDFS

In [14]:
# make files
client.upload('hdfs456', 'davinci_notebooks.txt', overwrite=True) # bestand bestaat al -> error -> check met behulp van status of het bestaat of gebruik overwrite parameter
client.upload('hdfs456/boek2.txt', 'outline_of_science.txt', overwrite=True) # als je een pad geeft naar iets dat niet bestaat, dan wordt het hernoemd naar de opgegeven naa
client.upload('hdfs456/boek3.txt', 'ulysses.txt', overwrite=True) # schrijf de file
client.write('hdfs456/boek4.txt', 'ulysses.txt', overwrite=True) # schrijf data

### Bekijken van het filesysteem

In [24]:
# hdfs dfs -ls command

print(client.status('hdfs456/boek4.txt'))
print(client.acl_status('hdfs456/boek4.txt'))
print(client.checksum('hdfs456/boek4.txt'))
print(client.list('hdfs456'))
print(client.walk('/'))

for f in client.walk('/'):
    print(f)

{'accessTime': 1740061883702, 'blockSize': 134217728, 'childrenNum': 0, 'fileId': 16436, 'group': 'supergroup', 'length': 11, 'modificationTime': 1740061883731, 'owner': 'bigdata', 'pathSuffix': '', 'permission': '644', 'replication': 3, 'storagePolicy': 0, 'type': 'FILE'}
{'entries': [], 'group': 'supergroup', 'owner': 'bigdata', 'permission': '644', 'stickyBit': False}
{'algorithm': 'MD5-of-0MD5-of-512CRC32C', 'bytes': '000002000000000000000000fb1d52cd3af9c4cf8e77e21537e9a41800000000', 'length': 28}
['boek2.txt', 'boek3.txt', 'boek4.txt', 'davinci_notebooks.txt']
<generator object Client.walk at 0x7ff209437680>
('/', ['rmstate', 'user'], [])
('/rmstate', ['FSRMStateRoot'], [])
('/rmstate/FSRMStateRoot', ['AMRMTokenSecretManagerRoot', 'ProxyCARoot', 'RMAppRoot', 'RMDTSecretManagerRoot', 'ReservationSystemRoot'], ['EpochNode', 'RMVersionNode'])
('/rmstate/FSRMStateRoot/AMRMTokenSecretManagerRoot', [], ['AMRMTokenSecretManagerNode'])
('/rmstate/FSRMStateRoot/ProxyCARoot', [], ['caCert',

## Uitlezen van het filesysteem

In [17]:
client.download('hdfs456/boek4.txt', 'test.txt', overwrite=True) # maakt een lokale file aan

with client.read('hdfs456/boek4.txt') as reader:
    content=reader.read()
    print(content)

b'ulysses.txt'


### Aanpassen van het filesysteem

In [29]:
#client.delete('hdfs', recursive=True)

#client.rename('hdfs456/boek2.txt', 'hdfs456/boek.txt') # dit kan trouwens ook gebruikt worden om te verplaatsen

client.set_permission('hdfs456/boek4.txt', 777)
client.set_replication('hdfs456/boek4.txt', 4)

### Oefening

Los de volgende oefeningen op:

Schrijf een Python-script dat de informatie van een bestand in HDFS ophaalt en weergeeft. De informatie moet onder andere de grootte van het bestand, de eigenaar, de groep en de permissies bevatten.

Schrijf een Python-script dat de volledige inhoud van een tekstbestand in HDFS leest en afdrukt naar de console.

Schrijf een Python-script dat een tekstbestand van de lokale schijf naar HDFS schrijft. Het script moet de inhoud van het lokale bestand lezen en deze naar een nieuw bestand in HDFS schrijven.

Schrijf een Python-script dat de permissies van een bestand in HDFS wijzigt. Het script moet de permissies instellen op een door de gebruiker opgegeven waarde (bijvoorbeeld 755).

Schrijf een Python-script dat de huidige replicatiefactor van een bestand in HDFS controleert en weergeeft.

Schrijf een Python-script dat alle bestanden in een HDFS-directory zoekt die voldoen aan een bepaald naamspatroon (bijvoorbeeld alle .txt bestanden).

Schrijf een Python-script dat nieuwe inhoud toevoegt aan een bestaand bestand in HDFS. Het script moet de nieuwe inhoud aan het einde van het bestand toevoegen.

Schrijf een Python-script dat de checksum van een bestand in HDFS ophaalt en weergeeft. Dit kan nuttig zijn om de integriteit van bestanden te controleren.


Schrijf python code dat controleert of een directory bestaat in het hdfs.
Indien nee wordt de directory aangemaakt.
Indien ja, worden alle files in de directory verwijderd om van een lege directory te starten.
Upload daarna een tekst-bestand naar keuze.